In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import shutil
from sklearn.model_selection import train_test_split
import random
import matplotlib.image as mpimg
from tensorflow.keras.applications import EfficientNetB0 # type: ignore
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization # type: ignore
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [14]:
train_dir = "/Users/michelangelozampieri/Desktop/image_classifier/data_output/train"
test_dir = "/Users/michelangelozampieri/Desktop/image_classifier/data_output/test"
validation_dir = "/Users/michelangelozampieri/Desktop/image_classifier/data_output/validation"

In [15]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest"
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode="categorical"
)

Found 4582 images belonging to 4 classes.


In [16]:
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode="categorical"
)

Found 1591 images belonging to 4 classes.


In [17]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode="categorical"
)

Found 1127 images belonging to 4 classes.


In [20]:
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False  # Initially freeze the base model

# Custom classification head
x = GlobalAveragePooling2D()(base_model.output)  # Reduce feature maps to a single vector
x = Dense(64, activation="relu")(x)  # Fewer neurons for simplicity
x = BatchNormalization()(x)  # Optional: Stabilize training
output_layer = Dense(4, activation="softmax")(x)  # Output layer

# Create model
model = Model(inputs=base_model.input, outputs=output_layer)

model.compile(optimizer="adam", 
              loss="categorical_crossentropy", 
              metrics=["accuracy"])

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
)

Epoch 1/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 39s 241ms/step - accuracy: 0.2744 - loss: 1.4427 - val_accuracy: 0.2678 - val_loss: 1.4505
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 32s 224ms/step - accuracy: 0.3364 - loss: 1.3473 - val_accuracy: 0.4261 - val_loss: 1.4379
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 32s 222ms/step - accuracy: 0.3927 - loss: 1.2854 - val_accuracy: 0.2678 - val_loss: 1.9938
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 33s 228ms/step - accuracy: 0.4382 - loss: 1.1882 - val_accuracy: 0.2678 - val_loss: 1.3011
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 33s 231ms/step - accuracy: 0.4567 - loss: 1.1444 - val_accuracy: 0.1967 - val_loss: 2.0892
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 34s 237ms/step - accuracy: 0.4829 - loss: 1.0495 - val_accuracy: 0.2678 - val_loss: 3.7677
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 36s 251ms/step - accuracy: 0.5036 - loss: 1.0298 - val_accuracy: 0.2678 - val_loss: 4.6198
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 38s 262ms/step - accuracy: 0.5341 - loss: 0

In [21]:
val_acc, val_loss = model.evaluate(test_generator)
print("Validation accuracy: ", val_acc)
print("Validation loss: ", val_loss)

36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 158ms/step - accuracy: 0.1958 - loss: 5.0480
Validation accuracy:  5.014168739318848
Validation loss:  0.20141969621181488


In [22]:
classes =['cloudy', 'desert', 'green_area', 'water']

def get_random_images(test_dir, num_images=5):
    random_images = []
    class_names = os.listdir(test_dir)
    for _ in range(num_images):
        random_class = random.choice(class_names)
        class_dir = os.path.join(test_dir, random_class)
        random_image = random.choice(os.listdir(class_dir))
        random_images.append((os.path.join(class_dir, random_image), random_class))
    return random_images

from tensorflow.keras.preprocessing.image import load_img, img_to_array #type: ignore

def predict_random_images(model, test_dir, num_images=5):
    random_images = get_random_images(test_dir, num_images)
    for img_path, true_class in random_images:
        # Load and preprocess the image
        img = load_img(img_path, target_size=(150, 150))
        img_array = img_to_array(img) / 255.0  # Normalize the image
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        
        # Make prediction
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions, axis=1)[0]
        
        return true_class, classes[predicted_class]
    
def evaluate_model(model, test_dir, num_images=1000):
    correct = 0
    for _ in range(num_images):
        true_class, predicted_class = predict_random_images(model, test_dir)
        if true_class == predicted_class:
            correct += 1
    return correct / num_images

In [23]:
print(evaluate_model(model, test_dir, num_images=1000))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 718ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━

In [24]:
model.save("efficienetb0_base_model.h5")

In [25]:
model.trainable = True
for layer in model.layers[:-20]:  # Freeze all layers except the last 20
    layer.trainable = False

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

model.compile(
    optimizer=optimizer, 
    loss="categorical_crossentropy", 
    metrics=["accuracy"])

history_fine = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

Epoch 1/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 40s 251ms/step - accuracy: 0.5484 - loss: 0.8785 - val_accuracy: 0.4670 - val_loss: 0.8133
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 33s 225ms/step - accuracy: 0.5436 - loss: 0.8811 - val_accuracy: 0.6103 - val_loss: 0.7814
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 32s 219ms/step - accuracy: 0.5576 - loss: 0.8781 - val_accuracy: 0.6197 - val_loss: 0.7889
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 32s 223ms/step - accuracy: 0.5652 - loss: 0.8571 - val_accuracy: 0.6549 - val_loss: 0.7672
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 32s 225ms/step - accuracy: 0.5700 - loss: 0.8913 - val_accuracy: 0.5996 - val_loss: 0.7783
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 33s 227ms/step - accuracy: 0.5759 - loss: 0.8440 - val_accuracy: 0.5971 - val_loss: 0.8151
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 33s 230ms/step - accuracy: 0.5601 - loss: 0.8966 - val_accuracy: 0.6417 - val_loss: 0.7767
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 34s 237ms/step - accuracy: 0.5656 - loss: 0

In [26]:
val_acc, val_loss = model.evaluate(test_generator)
print("Validation accuracy: ", val_acc)
print("Validation loss: ", val_loss)

36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 212ms/step - accuracy: 0.7165 - loss: 0.7538
Validation accuracy:  0.7584282755851746
Validation loss:  0.6974267959594727


In [27]:
print(evaluate_model(model, test_dir, num_images=1000))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 800ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━

In [28]:
model.save("fine_tuned_efficientnetb0.h5")

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)  # Pooling layer
x = Dense(256, activation='relu')(x)  # Fully connected layer
x = Dropout(0.5)(x)  # Dropout for regularization
predictions = Dense(4, activation='softmax')(x)  # Output layer

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator)
)

Epoch 1/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 86s 582ms/step - accuracy: 0.2762 - loss: 1.4892 - val_accuracy: 0.2684 - val_loss: 1.3784
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 87s 602ms/step - accuracy: 0.2866 - loss: 1.3794 - val_accuracy: 0.2678 - val_loss: 1.3760
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 87s 603ms/step - accuracy: 0.2707 - loss: 1.3775 - val_accuracy: 0.2678 - val_loss: 1.3819
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 96s 664ms/step - accuracy: 0.2731 - loss: 1.3803 - val_accuracy: 0.2684 - val_loss: 1.3777
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 98s 683ms/step - accuracy: 0.2596 - loss: 1.3828 - val_accuracy: 0.2678 - val_loss: 1.3794
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 100s 695ms/step - accuracy: 0.2626 - loss: 1.3790 - val_accuracy: 0.2678 - val_loss: 1.3800
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 102s 706ms/step - accuracy: 0.2716 - loss: 1.3801 - val_accuracy: 0.2678 - val_loss: 1.3792
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 102s 709ms/step - accuracy: 0.2737 - loss

In [32]:
val_acc, val_loss = model.evaluate(test_generator)
print("Validation accuracy: ", val_acc)
print("Validation loss: ", val_loss)

36/36 ━━━━━━━━━━━━━━━━━━━━ 18s 490ms/step - accuracy: 0.4407 - loss: 1.3786
Validation accuracy:  1.378175139427185
Validation loss:  0.4338952898979187


In [33]:
print(evaluate_model(model, test_dir, num_images=1000))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 726ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━

In [34]:
model.save("resnet50_base_model.h5")

In [35]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

for layer in base_model.layers:
    layer.trainable = False

In [36]:
def extract_features(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = base_model.predict(img_array)
    return features

def extract_features_for_dataset(dataset_dir):
    features = []
    labels = []
    for class_name in os.listdir(dataset_dir):
        class_dir = os.path.join(dataset_dir, class_name)
        if os.path.isdir(class_dir):
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                feature = extract_features(img_path)
                features.append(feature)
                labels.append(class_name)
    return np.array(features), np.array(labels)

In [37]:
# Extract features for training data
train_features, train_labels = extract_features_for_dataset(train_dir)

# Extract features for validation data
val_features, val_labels = extract_features_for_dataset(validation_dir)

# Reshape features (if necessary)
train_features = train_features.reshape(train_features.shape[0], -1)
val_features = val_features.reshape(val_features.shape[0], -1)

# Save the corresponding labels
train_labels = train_generator.classes
val_labels = validation_generator.classes

train_labels = to_categorical(train_labels, num_classes=train_generator.num_classes)
val_labels = to_categorical(val_labels, num_classes=validation_generator.num_classes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 609ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━

In [38]:
model = Sequential([
    Flatten(input_shape=train_features.shape[1:]),
    Dense(512, activation='relu', kernel_regularizer=l2(0.01)),  # L2 regularization
    Dropout(0.5),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax') # Output layer
])

# Compile the classifier
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [39]:
history = model.fit(
    train_features, train_labels,
    validation_data=(val_features, val_labels),
    epochs=10,
    batch_size=32,
)

Epoch 1/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 15s 102ms/step - accuracy: 0.7011 - loss: 10.2021 - val_accuracy: 0.8548 - val_loss: 5.2667
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 15s 106ms/step - accuracy: 0.7886 - loss: 4.9486 - val_accuracy: 0.8554 - val_loss: 3.3184
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 16s 110ms/step - accuracy: 0.8120 - loss: 3.2007 - val_accuracy: 0.8598 - val_loss: 2.4224
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 15s 106ms/step - accuracy: 0.8201 - loss: 2.5084 - val_accuracy: 0.8548 - val_loss: 2.2133
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 15s 106ms/step - accuracy: 0.8082 - loss: 2.3069 - val_accuracy: 0.8554 - val_loss: 1.7330
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 15s 104ms/step - accuracy: 0.8511 - loss: 1.6342 - val_accuracy: 0.8592 - val_loss: 1.3229
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 15s 103ms/step - accuracy: 0.8562 - loss: 1.2933 - val_accuracy: 0.8592 - val_loss: 1.2252
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 15s 100ms/step - accuracy: 0.8475 - loss: 

In [ ]:
val_acc, val_loss = model.evaluate(val_features, val_labels)
print("Validation accuracy: ", val_acc)
print("Validation loss: ", val_loss)

50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9218 - loss: 0.8277
Validation accuracy:  0.9378803968429565
Validation loss:  0.8554368615150452


In [42]:
model.save("resnet50_fine_tuned.h5")